In [8]:
def plot_time_dependent_basic_path_loss(gdf,para) :

    # folder
    # ===================================================
    if not os.path.exists('data/FigurePlot/plot_time_dependent_basic_path_loss' ):
        os.makedirs( 'data/FigurePlot/plot_time_dependent_basic_path_loss' )
        print('create directory : plot_time_dependent_basic_path_loss' )
        
    folder_path = 'data/FigurePlot/plot_time_dependent_basic_path_loss/'

    # function
    # ===================================================
    def state_map(x):
        if x == "LOS" : 
            state = 2
            return state
        if x == 'NLOS': 
            state = 1
            return state
        state = 0
        return state
    
    def color_map(x):
        if x == "LOS" : 
            color = '#55A868'
            return color
        if x == 'NLOS': 
            color = '#C44E52'
            return color
        color = '#4C72B0'
        return color
    
    # plot
    # ===================================================
    buffer_df = gdf
    
    location_list = buffer_df["location"].unique()
    for location in location_list :
    
        gdf = buffer_df[buffer_df['location']==location]
    
        SAT_list = gdf["SAT_name"].unique()
        for sat in SAT_list : 

            sat_gdf = gdf[gdf['SAT_name']==sat]

            sat_gdf["state"] = sat_gdf["State"].map(state_map)
            sat_gdf["color"] = sat_gdf["State"].map(color_map)
            sat_gdf['rise_set'] = np.nan

            sat_gdf = sat_gdf.reset_index(drop=True)
            OFR_index_list =  sat_gdf[sat_gdf['State']=='out_of_range'].index.values.tolist()

            list_set = []
            list_rise = []


            for index in OFR_index_list :
                try :
                    if sat_gdf.iloc[index+1,:]['State'] != 'out_of_range' :
                        sat_gdf['rise_set'][index+1]='rise'
                        list_rise.append(index+1)
                except :
                    #print('error : rise :',str(index))
                    pass
                try :
                    if sat_gdf.iloc[index-1,:]['State'] != 'out_of_range' :
                        sat_gdf['rise_set'][index-1]='set'
                        list_set.append(index-1)
                except :
                    #print('error : set  :',str(index))
                    pass


            np_set  = np.array(list_set)
            np_rise = np.array(list_rise)

            np_interval = np_set - np_rise
            list_interval = np_interval.tolist()
            np_list = np.asarray([list_rise,list_set])
            list_xlim = np_list.T.tolist()

            fig_num = len(list_set)

            fig, ax = plt.subplots(1,fig_num,
                                   sharex=True,
                                   sharey=True,
                                   figsize=(30,15),
                                   gridspec_kw={'width_ratios': list_interval})
            fig.subplots_adjust(bottom=0.22)
            plt.xticks([])
            plt.yticks([])

            fig.text(0.5, 0.04, 'Time', ha='center', va='center', fontsize = 30)
            fig.text(0.06, 0.5, 'Basic Path Loss (dB)', ha='center', va='center', rotation='vertical', fontsize = 30)
            #

            for i in range(0,len(list_set)):


                ax[i] = fig.add_subplot(1,fig_num,i+1)
                plt.xticks(rotation=90)

                if i != 0 :

                    plt.setp( ax[i].get_yticklabels(), visible=False)

                min_xlim = sat_gdf[sat_gdf.index==list_xlim[i][0]]['TIME']
                max_xlim = sat_gdf[sat_gdf.index==list_xlim[i][1]]['TIME']


                t = sat_gdf[sat_gdf['State']!='out_of_range']

                ylim_max = t['PL_b'+' '+str(para.Environment)+' '+str(para.Fc)].max()
                ylim_min = t['PL_b'+' '+str(para.Environment)+' '+str(para.Fc)].min()

                ax[i].set_ylim(150,350)
                ax[i].set_xlim(min_xlim,max_xlim)

                ax[i].spines['right'].set_visible(False)
                ax[i].spines['left'].set_visible(False)


                fliter_LOS  = (sat_gdf['State']=='LOS')
                fliter_NLOS = (sat_gdf['State']=='NLOS') 

                line_type = ['--','-.',':']
                line_count = 0

                ax[i].plot_date(sat_gdf['TIME'], sat_gdf['PL_b'+' '+str(para.Environment)+' '+str(para.Fc)], 
                            line_type[line_count], 
                            markersize=10,label = str(para.Environment)+' '+str(para.Fc))
                line_count = line_count+1

                sat_gdf_LOS = sat_gdf[fliter_LOS]
                ax[i].plot_date(sat_gdf_LOS['TIME'], sat_gdf_LOS['PL_b'+' '+str(para.Environment)+' '+str(para.Fc)], 
                                '.', markersize=15,
                                c = '#55A868')
                sat_gdf_NLOS = sat_gdf[fliter_NLOS]
                ax[i].plot_date(sat_gdf_NLOS['TIME'], sat_gdf_NLOS['PL_b'+' '+str(para.Environment)+' '+str(para.Fc)], 
                                '.', markersize=15,
                                c = '#C44E52')

                nOFR_gdf = sat_gdf[sat_gdf['State']!='out_of_range']
                nOFR_gdf = nOFR_gdf.reset_index(drop=True)
                
                #pts = int(fc*10e9*30)
                pts = 100                              #每個取樣點間的模擬增加數量
                for index, row in nOFR_gdf.iterrows():

                    if index == nOFR_gdf.index.max() :
                        break

                    x_start = nOFR_gdf.loc[index, 'TIME']
                    x_end   = nOFR_gdf.loc[index+1, 'TIME']

                    y_start = nOFR_gdf.loc[index  , 'PL_b'+' '+str(para.Environment)+' '+str(para.Fc)]
                    y_end   = nOFR_gdf.loc[index+1, 'PL_b'+' '+str(para.Environment)+' '+str(para.Fc)]

                    sf_start = nOFR_gdf.loc[index  , 'SF_sigma'+' '+str(para.Environment)+' '+str(para.Fc)]
                    sf_end   = nOFR_gdf.loc[index+1, 'SF_sigma'+' '+str(para.Environment)+' '+str(para.Fc)]

                    sf_line = np.linspace(sf_start, sf_end, pts)

                    def rand_normal(num):
                        return np.random.normal(0, num, 1)[0]
                        #return np.random.lognormal(0, num, 1)[0]
                        #雖然SF是lognormal 分布
                        #但當換至dB時會成為normal 分布
                        #因此這裡用normal 才是對的
                        
                    apply_rand_normal = np.vectorize(rand_normal)
                    noise = apply_rand_normal(sf_line)

                    x = pd.date_range(start=x_start,
                                  end=x_end,
                                  periods=pts)
                    y = np.linspace(y_start, y_end, pts)

                    ax[i].plot_date(x, y+noise, '-',c='black')


                # 斷軸
                # ========================================================================
                y_d = .03
                x_d = 1/list_interval[i]
                # how big to make the diagonal lines in axes coordinates
                # arguments to pass to plot, just so we don't keep repeating them
                kwargs = dict(transform=ax[i].transAxes, color='k', clip_on=False)
                if i != len(list_set)-1 :
                    ax[i].plot((1 - x_d, 1 + x_d), (1 - y_d, 1 + y_d), **kwargs)  # bottom-right diagonal
                    ax[i].plot((1 - x_d, 1 + x_d), (-y_d, +y_d), **kwargs)        # top-right diagonal
                if i != 0 :
                    ax[i].plot((-x_d, +x_d), (1 - y_d, 1 + y_d), **kwargs)        # bottom-left diagonal
                    ax[i].plot((-x_d, +x_d), (-y_d, +y_d), **kwargs)              # top-left diagonal
                # ========================================================================


                timeFmt = mdates.DateFormatter('%b %d, %H:%M')
                ax[i].xaxis.set_major_locator(mdates.MinuteLocator())
                ax[i].xaxis.set_major_formatter(timeFmt)

                my_xticks = ax[i].get_xticks()
                ax[i].set_xticks([my_xticks[0], my_xticks[-1]],rotation=90)
                ax[i].tick_params(axis='both', which='major', labelsize=24)

            plt.savefig(folder_path + str(sat) + '_' + str(location) + '_' + str(para.Environment) + '_' + str(para.Fc) + '.png')
            plt.close()